<a href="https://colab.research.google.com/github/Franco-Urrutia-G/EIE401-1s-2024/blob/main/MIE865%20MicroGridOptimitation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cplex

In [5]:
from cplex import Cplex
from cplex.exceptions import CplexError
import cplex

def resolver_problema(precio_compra_electricidad, precio_venta_electricidad, precio_compra_calor, precio_venta_calor, demanda_electricidad, demanda_calor):
    # Crear una instancia del problema
    prob = Cplex()
    prob.set_problem_type(Cplex.problem_type.LP)

    # Silenciar el output
    prob.set_log_stream(None)
    prob.set_error_stream(None)
    prob.set_warning_stream(None)
    prob.set_results_stream(None)

    # Nombres de las variables
    variable_names = ["DGA", "DGB", "CHPA", "CHPB", "HOB", "B_E", "B_H", "S_E", "S_H"]

    # Límites inferiores y superiores de las variables
    lower_bounds = [0, 0, 30, 50, 0, 0, 0, 0, 0]
    upper_bounds = [100, 80, 60, 100, 80, cplex.infinity, cplex.infinity, cplex.infinity, cplex.infinity]

    # Agregar variables
    prob.variables.add(names=variable_names, lb=lower_bounds, ub=upper_bounds)

    # Coeficientes de la función objetivo para costos y ventas
    precio_calor_generado = 80
    precio_electricidad_generada = [135, 140, 150, 145]

    # Definir la función objetivo compuesta
    objective = [
        precio_electricidad_generada[0],  # DGA
        precio_electricidad_generada[1],  # DGB
        precio_electricidad_generada[2],  # CHPA
        precio_electricidad_generada[3],  # CHPB
        precio_calor_generado,            # HOB
        precio_compra_electricidad,       # B_E
        precio_compra_calor,              # B_H
        -precio_venta_electricidad,       # S_E
        -precio_venta_calor               # S_H
    ]

    # Establecer la función objetivo
    prob.objective.set_sense(prob.objective.sense.minimize)
    prob.objective.set_linear(zip(variable_names, objective))

    # Restricciones

    # Restricción de demanda de electricidad
    constraint_names = ["Electricity_Demand"]
    constraint_expr = [
        [["DGA", "DGB", "CHPA", "CHPB", "B_E", "S_E"], [1, 1, 1, 1, 1, -1]]
    ]
    rhs = [demanda_electricidad]  # Demanda de electricidad
    senses = ["E"]

    # Agregar la restricción de demanda de electricidad
    prob.linear_constraints.add(names=constraint_names, lin_expr=constraint_expr, rhs=rhs, senses=senses)

    # Restricción de demanda de calor
    constraint_names = ["Heat_Demand"]
    constraint_expr = [
        [["CHPA", "CHPB", "HOB", "B_H", "S_H"], [1.2, 0.8, 1, 1, -1]]
    ]
    rhs = [demanda_calor]  # Demanda de calor
    senses = ["E"]

    # Agregar la restricción de demanda de calor
    prob.linear_constraints.add(names=constraint_names, lin_expr=constraint_expr, rhs=rhs, senses=senses)

    # Resolver el problema
    try:
        prob.solve()
    except CplexError as exc:
        print(exc)

    # Obtener los resultados
    solution_values = prob.solution.get_values()

    # Calcular la generación de calor para CHP A y CHP B
    heat_A = 1.2 * solution_values[2]  # CHPA
    heat_B = 0.8 * solution_values[3]  # CHPB

    # Calcular los costos totales de electricidad y calor
    costo_electricidad_generada = sum(precio_electricidad_generada[i] * solution_values[i] for i in range(4))
    costo_calor_generado = precio_calor_generado * solution_values[4]
    costo_electricidad_comprada = precio_compra_electricidad * solution_values[5]
    costo_calor_comprado = precio_compra_calor * solution_values[6]
    venta_electricidad = precio_venta_electricidad * solution_values[7]
    venta_calor = precio_venta_calor * solution_values[8]

    costo_total_electricidad = costo_electricidad_generada + costo_electricidad_comprada - venta_electricidad
    costo_total_calor = costo_calor_generado + costo_calor_comprado - venta_calor
    costo_total_minimizado = prob.solution.get_objective_value()

    # Imprimir los resultados
    for name, value in zip(variable_names, solution_values):
        print(f"{name}: {value} kWh")

    print(f"CHPA_Heat: {heat_A} kWh")
    print(f"CHPB_Heat: {heat_B} kWh")
    print(f"Costo total electricidad: {costo_total_electricidad} Won")
    print(f"Costo total calor: {costo_total_calor} Won")
    print(f"Costo total minimizado: {costo_total_minimizado} Won")

# Definir vectores de valores para iterar
precios_compra_electricidad = [138, 139, 143, 149, 150]
precios_venta_electricidad = [130, 131, 134, 138, 139]
precios_compra_calor = [77, 77, 77, 77, 77]
precios_venta_calor = [75, 75, 75, 75, 75]
demandas_electricidad = [183, 183.5, 190, 192, 223]
demandas_calor = [130, 125, 120, 122, 130]

# Iterar
for precio_compra_e, precio_venta_e, precio_compra_h, precio_venta_h, demanda_e, demanda_h in zip(
    precios_compra_electricidad, precios_venta_electricidad, precios_compra_calor, precios_venta_calor, demandas_electricidad, demandas_calor):
    print(f"\nResolviendo para:")
    print(f"Precio compra electricidad: {precio_compra_e}, Precio venta electricidad: {precio_venta_e}")
    print(f"Precio compra calor: {precio_compra_h}, Precio venta calor: {precio_venta_h}")
    print(f"Demanda electricidad: {demanda_e}, Demanda calor: {demanda_h}")
    resolver_problema(precio_compra_e, precio_venta_e, precio_compra_h, precio_venta_h, demanda_e, demanda_h)



Resolviendo para:
Precio compra electricidad: 138, Precio venta electricidad: 130
Precio compra calor: 77, Precio venta calor: 75
Demanda electricidad: 183, Demanda calor: 130
DGA: 23.0 kWh
DGB: 0.0 kWh
CHPA: 60.0 kWh
CHPB: 100.0 kWh
HOB: 0.0 kWh
B_E: 0.0 kWh
B_H: 0.0 kWh
S_E: 0.0 kWh
S_H: 22.0 kWh
CHPA_Heat: 72.0 kWh
CHPB_Heat: 80.0 kWh
Costo total electricidad: 26605.0 Won
Costo total calor: -1650.0 Won
Costo total minimizado: 24955.0 Won

Resolviendo para:
Precio compra electricidad: 139, Precio venta electricidad: 131
Precio compra calor: 77, Precio venta calor: 75
Demanda electricidad: 183.5, Demanda calor: 125
DGA: 23.5 kWh
DGB: 0.0 kWh
CHPA: 60.0 kWh
CHPB: 100.0 kWh
HOB: 0.0 kWh
B_E: 0.0 kWh
B_H: 0.0 kWh
S_E: 0.0 kWh
S_H: 27.0 kWh
CHPA_Heat: 72.0 kWh
CHPB_Heat: 80.0 kWh
Costo total electricidad: 26672.5 Won
Costo total calor: -2025.0 Won
Costo total minimizado: 24647.5 Won

Resolviendo para:
Precio compra electricidad: 143, Precio venta electricidad: 134
Precio compra calor: 77